In [69]:
#!pip install twython

In [70]:
import re
import string
import csv
import nltk

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *
from nltk.metrics import ConfusionMatrix
from sklearn.model_selection import train_test_split

import sqlite3
from sqlite3 import Error

In [71]:
def processSentence(sentence):
    # process the tweets

    #Convert to lower case
    sentence = sentence.lower()
    #Convert www.* or https?://* to URL
    sentence = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',sentence)
    #Convert @username to AT_USER
    sentence = re.sub('@[^\s]+','AT_USER',sentence)
    #Remove additional white spaces
    sentence = re.sub('[\s]+', ' ', sentence)
    #Replace #word with word
    sentence = re.sub(r'#([^\s]+)', r'\1', sentence)
    #trimd
    sentence = sentence.strip('\'"')
    #*************************
    #sentence = sentence.replace(" q ", " que ")    
    return sentence

In [72]:
#initialize stopWords
stopWords = []

#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    
    if 1==1:
        stopWords = nltk.corpus.stopwords.words('portuguese')
    else:
        fp = open(stopWordListFileName, 'r')
        line = fp.readline()
        while line:
            word = line.strip()
            stopWords.append(word)
            line = fp.readline()
        fp.close()
        
    #****************************************
    stopWords.append('AT_USER')
    stopWords.append('URL')
    stopWords.append('pra')
    #*****************************************
            
    return stopWords
#end

In [73]:
#start getfeatureVector
def getFeatureVector(sentence,stopWords):
    featureVector = []
    #split tweet into words
    words = sentence.split()
    for w in words:        
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        if len(w)>0:
            if 1==1:
                #print(len(w))
                aux = w
                stemmer = nltk.stem.SnowballStemmer('portuguese')
                w = stemmer.stem(w)
                #if w=='indo':
                    #print(aux)
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
        
    return featureVector
#end


In [74]:
def extract_features(sentence):
    sentence_words = set(sentence)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in sentence_words)
    return features
#end

In [75]:
#Read the sentence one by one and process it
inpTweets = csv.reader(open('BaseReLi2.csv', 'r'), delimiter=',')

stopWords = getStopWordList('portuguese')
featureList = []
featureVectorPositive=[]
featureVectorNonPositive=[]

# Get tweet words
cont_neg=0
cont_pos=0
cont_neu=0
cont_max=3000
item_vec_max = 1

for row in inpTweets:
    if len(row)==4:
        if row[3] == '1' and cont_pos <= cont_max:
            cont_pos +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                featureVectorPositive.append(" ".join(featureVector))
                1==1
            else:
                #print(featureVector, sentiment)
                1==1

        elif row[3] == '-1' and cont_neg <= cont_max:
            cont_neg +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                featureVectorNonPositive.append(" ".join(featureVector))
                1==1
            else:
                #print(featureVector, sentiment)
                1==1

        elif row[3] == '0' and cont_neu <= cont_max:
            cont_neu +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                featureVectorPositive.append(" ".join(featureVector))
                1==1
            else:
                #print(featureVector, sentiment)
                1==1
#end loop

In [76]:
def features(sentence):
    words = sentence.lower().split()
    d={}
    for w in words:
        d['contains(%s)' % w]=True
        
    return d

In [77]:
featureVectorNonPositive = list(map(features, featureVectorNonPositive))

In [78]:
featureVectorPositive = list(map(features, featureVectorPositive))

In [79]:
FVP_train ,FVP_test = train_test_split(featureVectorPositive,random_state=24061976,shuffle=True,test_size=0.10)
FVnP_train ,FVnP_test = train_test_split(featureVectorNonPositive,random_state=24061976,shuffle=True,test_size=0.10)

NAIVE BAYES CLASSIFIER

In [80]:
from nltk.classify import PositiveNaiveBayesClassifier
classifier = PositiveNaiveBayesClassifier.train(featureVectorPositive,FVnP_train)

In [81]:
test = '#BTSBBMAs to indo pra quarta conta ainda :('
processedTestTweet = processSentence(test)
test = " ".join((getFeatureVector(processedTestTweet,stopWords)))
print(test)
classifier.classify(features(test))

btsbbmas to indo quart cont aind


False

In [49]:
print(classifier.show_most_informative_features(10000))

Most Informative Features
          contains(frac) = True            False : True   =     73.4 : 1.0
        contains(entedi) = True            False : True   =     51.8 : 1.0
         contains(brilh) = True            False : True   =     48.3 : 1.0
       contains(preguic) = True            False : True   =     46.4 : 1.0
      contains(bonitinh) = True            False : True   =     46.4 : 1.0
        contains(obsess) = True            False : True   =     46.4 : 1.0
       contains(criatur) = True            False : True   =     46.4 : 1.0
         contains(irrit) = True            False : True   =     41.2 : 1.0
        contains(detalh) = True             True : False  =     39.7 : 1.0
     contains(decepcion) = True            False : True   =     36.0 : 1.0
           contains(lix) = True            False : True   =     35.5 : 1.0
      contains(insuport) = True            False : True   =     35.5 : 1.0
           contains(bob) = True            False : True   =     34.9 : 1.0

          contains(body) = True            False : True   =      5.8 : 1.0
       contains(distorc) = True            False : True   =      5.8 : 1.0
       contains(corrent) = True            False : True   =      5.8 : 1.0
          contains(perf) = True            False : True   =      5.8 : 1.0
        contains(rabusc) = True            False : True   =      5.8 : 1.0
      contains(supremac) = True            False : True   =      5.8 : 1.0
        contains(intrus) = True            False : True   =      5.8 : 1.0
       contains(precios) = True            False : True   =      5.8 : 1.0
      contains(rotineir) = True            False : True   =      5.8 : 1.0
          contains(forj) = True            False : True   =      5.8 : 1.0
        contains(ostent) = True            False : True   =      5.8 : 1.0
         contains(gangu) = True            False : True   =      5.8 : 1.0
        contains(licenc) = True            False : True   =      5.8 : 1.0
          contains(vexx) 

          contains(desc) = True             True : False  =      1.8 : 1.0
           contains(peg) = True             True : False  =      1.8 : 1.0
        contains(revers) = True             True : False  =      1.8 : 1.0
       contains(complex) = True             True : False  =      1.8 : 1.0
     contains(entrelinh) = True             True : False  =      1.8 : 1.0
         contains(quadr) = True             True : False  =      1.8 : 1.0
        contains(inspir) = True             True : False  =      1.8 : 1.0
        contains(distop) = True             True : False  =      1.8 : 1.0
         contains(inser) = True             True : False  =      1.8 : 1.0
           contains(ton) = True             True : False  =      1.8 : 1.0
      contains(enriquec) = True             True : False  =      1.8 : 1.0
      contains(interior) = True             True : False  =      1.8 : 1.0
         contains(colet) = True             True : False  =      1.8 : 1.0
         contains(espac) 

         contains(atrat) = None             True : False  =      1.0 : 1.0
    contains(mulherzinh) = None             True : False  =      1.0 : 1.0
     contains(taquicard) = None             True : False  =      1.0 : 1.0
          contains(manh) = None             True : False  =      1.0 : 1.0
        contains(licenc) = None             True : False  =      1.0 : 1.0
          contains(sabn) = None             True : False  =      1.0 : 1.0
      contains(rotineir) = None             True : False  =      1.0 : 1.0
    contains(classifiqu) = None             True : False  =      1.0 : 1.0
            contains(tb) = None             True : False  =      1.0 : 1.0
       contains(coletiv) = None             True : False  =      1.0 : 1.0
     contains(descontra) = None             True : False  =      1.0 : 1.0
       contains(insting) = None             True : False  =      1.0 : 1.0
       contains(bomzinh) = None             True : False  =      1.0 : 1.0
     contains(entristec) 

        contains(tamanh) = None             True : False  =      1.0 : 1.0
        contains(roteir) = None             True : False  =      1.0 : 1.0
         contains(centr) = None             True : False  =      1.0 : 1.0
          contains(conv) = None             True : False  =      1.0 : 1.0
        contains(soberb) = None             True : False  =      1.0 : 1.0
          contains(apag) = None             True : False  =      1.0 : 1.0
      contains(justific) = None             True : False  =      1.0 : 1.0
        contains(essenc) = None             True : False  =      1.0 : 1.0
         contains(subit) = None             True : False  =      1.0 : 1.0
        contains(primor) = None             True : False  =      1.0 : 1.0
          contains(eleg) = None             True : False  =      1.0 : 1.0
         contains(divin) = None             True : False  =      1.0 : 1.0
        contains(decifr) = None             True : False  =      1.0 : 1.0
      contains(gravidez) 

In [ ]:
test_result = []
gold_result = []

for i in range(0,len(FVnP_test)):
    #test_result.append(classifier.classify(FVP_test[i])
    test_result.append(str(classifier.classify(FVnP_test[i])))
    gold_result.append('False')
    if test_result != gold_result:
        dict = {}
        dict = FVnP_test[i]
        list_key = list(dict.keys())
        print("Classificação: " + 'True')
        print("Palavras".center(70,"*"))
        for c in range(0,len(list_key)):
            if dict.get(list_key[c])==True:
                print(list_key[c]) 
                
        print("Fim".center(70,"*"))

CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)    

In [ ]:
labels = {'True', 'False'}

from collections import Counter
TP, FN, FP = Counter(), Counter(), Counter()
for i in labels:
    for j in labels:
        if i == j:
            TP[i] += int(CM[i,j])
        else:
            FN[i] += int(CM[i,j])
            FP[j] += int(CM[i,j])

print("label\tprecision\trecall\tf_measure")
for label in sorted(labels):
    precision, recall = 0, 0
    if TP[label] == 0:
        f_measure = 0
    else:
        precision = float(TP[label]) / (TP[label]+FP[label])
        recall = float(TP[label]) / (TP[label]+FN[label])
        f_measure = float(2) * (precision * recall) / (precision + recall)
    print(label+"\t"+str(precision)+"\t"+str(recall)+"\t"+str(f_measure))
    